<a href="https://colab.research.google.com/github/ch-seungyeon/A-free_market/blob/master/EfficientNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%cd /content/drive/My Drive/Colab Notebooks/ai_hub

/content/drive/My Drive/Colab Notebooks/ai_hub


In [ ]:
import numpy as np
import pandas as pd

# 2. 데이터 불러오기

In [ ]:
NIPA_256 = np.load('./NIPA_256.npz')

In [ ]:
train = NIPA_256['X_train']
# test = NIPA_256['X_test']
train_class = NIPA_256['y_train']

In [ ]:
print(train.shape)
# print(test.shape)
print(train_class.shape)

(16000, 256, 256, 3)
(16000, 20)


In [ ]:
# train_plant_cat = pd.get_dummies(train_plant).values

In [ ]:
# train_class_cat = pd.get_dummies(train_class).values

In [ ]:
X_train = train[:12800]
X_validation = train[12800:]
y_train = train_class[:12800]
y_validation = train_class[12800:]
print(X_train.shape, X_validation.shape)
print(y_train.shape, y_validation.shape)

(12800, 256, 256, 3) (3200, 256, 256, 3)
(12800, 20) (3200, 20)


In [ ]:
# y_train_plant_cat = train_plant_cat[:12800]
# y_validation_plant_cat = train_plant_cat[12800:]

In [ ]:
# y_train_class_cat = train_class_cat[:12800]
# y_validation_class_cat = train_class_cat[12800:]

# 3. 모델

“EfficientNet: Rethinking Model Scaling for Convolutional Neural Networks”(2019)

![](https://hoya012.github.io/assets/img/efficientnet/1.PNG)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications import EfficientNetB6
from tensorflow_addons.metrics import F1Score

In [ ]:
# EfficientNetB6().summary()

174465024/174460376 [==============================] - 2s 0us/step
Model: "efficientnetb6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 528, 528, 3) 0                                            
__________________________________________________________________________________________________
rescaling (Rescaling)           (None, 528, 528, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
normalization (Normalization)   (None, 528, 528, 3)  7           rescaling[0][0]                  
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 529, 529, 3)  0           normalization[0][0]              
__________________

## TEST1 imagenet weight의 필요성, 데이터확장X

In [ ]:
model_w = Sequential()
model_w.add(EfficientNetB6(include_top=False, input_shape=(256,256,3)))
model_w.add(GlobalAveragePooling2D())
model_w.add(Dropout(0.5))
model_w.add(Dense(20, activation='softmax'))

model_w.compile(loss='categorical_crossentropy',
                      optimizer='adam',
                      metrics=[F1Score(num_classes=20)])
model_w.summary()

earlystop = EarlyStopping(monitor='val_loss', patience=10)
model_w.fit(X_train, y_train, batch_size=8,
                  epochs=200,
                  validation_data=(X_validation, y_validation),
                  callbacks=[earlystop])

165240832/165234480 [==============================] - 1s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnetb6 (Functional)  (None, 8, 8, 2304)        40960143  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2304)              0         
_________________________________________________________________
dropout (Dropout)            (None, 2304)              0         
_________________________________________________________________
dense (Dense)                (None, 20)                46100     
Total params: 41,006,243
Trainable params: 40,781,804
Non-trainable params: 224,439
_________________________________________________________________
Epoch 1/200
1600/1600 [==============================] - 545s 341ms/step - loss: 0.5171 - f1_score: 0.8369 - val_loss: 0.2826 - val_f1_score: 0.9167
Epoch 2/200
1600/160

In [ ]:
model_plant_w = Sequential()
model_plant_w.add(EfficientNetB7(include_top=False, input_shape=(256,256,3)))
model_plant_w.add(GlobalAveragePooling2D())
model_plant_w.add(Dropout(0.5))
model_plant_w.add(Dense(20, activation='softmax'))

model_plant_w.compile(loss='categorical_crossentropy',
                      optimizer='adam',
                      metrics=[F1Score(num_classes=20)])
model_plant_w.summary()

earlystop = EarlyStopping(monitor='val_loss', patience=10)
model_plant_w.fit(X_train, y_train_class_cat, batch_size=8,
                  epochs=200,
                  validation_data=(X_validation, y_validation_class_cat),
                  callbacks=[earlystop])

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnetb7 (Functional)  (None, 8, 8, 2560)        64097687  
_________________________________________________________________
global_average_pooling2d_12  (None, 2560)              0         
_________________________________________________________________
dropout_12 (Dropout)         (None, 2560)              0         
_________________________________________________________________
dense_12 (Dense)             (None, 20)                51220     
Total params: 64,148,907
Trainable params: 63,838,180
Non-trainable params: 310,727
_________________________________________________________________
Epoch 1/200
1600/1600 [==============================] - 452s 283ms/step - loss: 0.5736 - f1_score: 0.8200 - val_loss: 0.4772 - val_f1_score: 0.8787
Epoch 2/200
1600/1600 [==============================] - 449s 280ms/step - loss: 0.2

In [ ]:
model_plant_nw = EfficientNetB7(weights=None, input_shape=(256,256,3), classes=8)

In [ ]:
model_plant_w = Sequential()
model_plant_w.add(EfficientNetB7(include_top=False, input_shape=(256,256,3)))
model_plant_w.add(GlobalAveragePooling2D())
model_plant_w.add(Dropout(0.5))
model_plant_w.add(Dense(8, activation='softmax'))

model_plant_w.compile(loss='sparse_categorical_crossentropy',
                      optimizer='adam',
                      metrics=['f1_score'])
model_plant_w.summary()

earlystop = EarlyStopping(monitor='val_loss', patience=10)
model.fit(data_augmentation.flow(X_train_112_nor, y_train_112, batch_size=128),
          steps_per_epoch=len(X_train_112_nor)/128,
          epochs=200,
          validation_data=(X_validation_112_nor, y_validation_112),
          callbacks=[earlystop])

In [ ]:
model_plant_w.summary()

In [ ]:
model_plant_nw = EfficientNetB7(weights=None, input_shape=(256,256,3), classes=8)

In [ ]:
model_plant_nw.summary()